<a href="https://colab.research.google.com/github/vvanvo/COOP-SQLPractice/blob/main/notebooks/COOP_SQL_103_PracticeNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SQL 103 - Combining Data From Multiple Tables**
Author: Martin Arroyo

### **About this notebook**

All of your queries will be written using preloaded databases that are available only in this notebook. Our "RDBMS" and SQL dialect is called `duckdb`, a new and popular Python library that provides the framework to make our queries possible. You can find [the documentation for `duckdb` here](https://duckdb.org/docs/sql/introduction) - you will want to keep the documentation handy.

`teachdb`, which provides the data that you will be working with, is a Python library written by The Freestack Initiative, a group of COOP alumni who want to empower the community to learn and improve their technical skills by providing materials and resources at low (or no) cost.

## **How to use this notebook**

First, we'll do a quick tutorial on how to use the notebook with these tools, then we'll dive into more SQL!

### **Step 1: Press the play button below to set up the database and notebook**

You will see a checkmark appear when the database is finished setting up.

In [1]:
%%capture --no-stderr
# @title Press Play { display-mode: "form" }

# This code is used to set up the notebook by installing the libraries we need, configuring extensions to
# make displays for our queries look nice, and connecting to our relational database so that you can write
# queries in code cells using the %%sql magic tag.

# Install `teachdb` if it's not in the system already
%pip install --quiet --upgrade git+https://github.com/freestackinitiative/teachingdb.git
from teachdb.teachdb import connect_teachdb
# Set configurations for notebook & load data
con = connect_teachdb(database=["core", "restaurant"])
%sql con

# Check out the Freestack Initiative @https://github.com/freestackinitiative

### **Step 2: Run a query**

To run SQL queries against the database, create a new code cell. Then write `%%sql` at the top. This tells the notebook that this cell is being used to query the database. You can write your queries underneath the `%%sql` line by pressing the play button of the cell or selecting the cell and using `CTRL + Shift + Enter` on Windows (`CMD + Shift + Return` on Mac.)

Go ahead and try it by executing the query in the cell below:

In [2]:
%%sql

SELECT *
FROM Customers
LIMIT 5

,CustomerID,FirstName,LastName,Email,Address,City,State,Phone,Birthday,FavoriteDish
0,1,Maggi,Domney,mdomney0@wisdompets.com,53177 Fieldstone Pass,San Bernardino,CA,760-702-5469,1938-10-11,15
1,2,Javier,Dawks,jdawks1@red30design.com,25629 Brown Trail,Hartford,CT,860-906-1459,1953-11-21,13
2,3,Aleen,Fasey,afasey2@kinetecoinc.com,41967 Mockingbird Court,Boca Raton,FL,561-410-2222,1900-08-10,3
3,4,Taylor,Jenkins,tjenkins@rouxacademy.org,968 Bartillon Park,Fort Lauderdale,FL,954-555-7424,1961-05-02,8
4,5,Imogen,Kabsch,ikabsch@landonhotel.com,222 Hudson Point,Anderson,SC,864-326-7456,1919-08-27,17


## **Combining Data - Joins and Unions**

Now we're ready to take our queries to the next level by combining data from multiple tables!

In this notebook, we'll walk you through how to combine data *horizontally* using joins and *vertically* with unions. At the end of it, you'll know:

- The two primary ways that we combine data in SQL
- How to write queries that combine data from multiple tables both *horizontally* and *vertically*

### Combining Data <em>Vertically</em> with `UNION`

Unions let us combine two tables together vertically. They say, "Combine this data from top to bottom." In other words, unions combine tables such that we add more rows to our data.

Here's what that looks like visually:

![Simple Union Results](https://github.com/freestackinitiative/coop_sql_notebooks/blob/v2/assets/simple-union.png?raw=1)

#### **Example - Using `UNION`**

**Unions are best to use when you have data from two very similar tables that can be combined into a single view.**

Our restaurant database has a `Dishes` table that has the main menu items and a `NewDishes` table which has dishes that are NOT in the menu yet.

We can use `UNION` to show us all the current and new dishes together instead of in separate tables. This will let us get a more holistic view of the restaurant’s offerings.

Here is the query that will show us all of the appetizers available from both the `Dishes` table and the `NewDishes` table together:

```sql
SELECT Name, Price, Type, 'NewDishes' AS TableName
FROM NewDishes
WHERE Type='Appetizer'
UNION
SELECT Name, Price, Type, 'Dishes' AS TableName
FROM Dishes
WHERE Type='Appetizer'
```

Copy this query into the cell below and run it to see the results:

In [3]:
%%sql

SELECT Name, Price, Type, 'NewDishes' AS TableName
FROM NewDishes
WHERE Type='Appetizer'
UNION
SELECT Name, Price, Type, 'Dishes' AS TableName
FROM Dishes
WHERE Type='Appetizer'

,Name,Price,Type,TableName
0,Parmesan Deviled Eggs,8.00,Appetizer,Dishes
1,Panko Stuffed Mushrooms,7.00,Appetizer,Dishes
2,Stuffed Bell Peppers,7.00,Appetizer,NewDishes
3,Pumpkin Hummus Platter,9.99,Appetizer,NewDishes
4,Artichokes with Garlic Aioli,9.00,Appetizer,Dishes
5,Barbecued Tofu Skewers,9.99,Appetizer,Dishes
6,Avocado Pesto Bruschetta,8.00,Appetizer,NewDishes
7,Zucchini Fritters,9.00,Appetizer,NewDishes


#### **Breakdown - Using `UNION`**

Let’s break down what’s happening in this query step-by-step:

##### **The "Top Query" in `UNION`**

First, you might notice that we’re actually using two separate queries. This is exactly how we write `UNION` queries - by combining the results of two queries on top of each other. We’ll start with the first part:

```sql
SELECT Name, Price, Type, 'NewDishes' AS TableName
FROM NewDishes
WHERE Type='Appetizer'
```

We’ll call this part of the query the “top query.” It is simply a `SELECT` query on a single table with a filter. We are querying the `NewDishes` table and filtering for only the `Appetizer` dishes.

It’s important to note both the columns that are being selected and their order - this is critical for `UNION` queries. For the results of both the top and bottom queries to align, the columns you select must have the same data type and be in the same order in both queries. You’ll see that we use columns with the same data type and in the same order for the “bottom query.”

Also, each query (both top and bottom) must select the same number of columns as well. You can’t have one query select five columns and the other select four - that is an error.

>**Extra Context:** The `'NewDishes' AS TableName` part creates a new column in our query called `TableName` that uses the text value `'NewDishes'` for all of its rows.


##### **The `UNION` clause**

```sql
UNION
```

Here is our new clause - `UNION`. By including `UNION` after our “top query”, we are saying, <em>“Combine the results from the query on top with the results from the query on the bottom.”</em>

Another important distinction to make here is that by saying `UNION`, we are also telling SQL that we don’t want duplicate values in our final query result. If we instead didn’t need to remove duplicates, we could use `UNION ALL` instead, which leaves in the duplicate rows between the top and bottom query.

>**Pro-Tip**: While using just `UNION` is handy when we don’t want duplicate rows in our results, it can cause performance issues when we are working with larger data sets. It is similar to the `DISTINCT` operation and should be used sparingly. If performance becomes an issue (e.g. query takes too long to run), switch to using `UNION ALL` - you’ll get duplicate rows but your query will run faster. Whether or not to make the tradeoff on performance vs. convenience is a judgement call you will have to make depending on your situation.

##### **The "Bottom Query" in `UNION`**

```sql
SELECT Name, Price, Type, 'Dishes' AS TableName
FROM Dishes
WHERE Type='Appetizer'
```

We’ll call this part the “bottom query”. In it, we’re saying, “Give me all the dishes in the Dishes table that aren’t appetizers” since we want to replace the current appetizers with the ones found in NewDishes. Again, we select columns that match both the data type and order of the columns from the “top query”.

#### **Summary - `UNION` queries**

To sum everything on `UNION` queries up so far:

- `UNION` queries allow us to combine data vertically instead of horizontally (like joins)
- To write `UNION` queries, we combine the results of two queries using the `UNION` clause
- The two queries must follow these rules:
    - The columns in both queries must have the same data types
    - The columns in both queries must be in the same order in each
- Each query must select the same number of columns
- `UNION` by itself tells SQL to remove duplicate values in the final result; `UNION ALL` keeps the duplicate values
- `UNION` is a more “expensive” operation than `UNION ALL` and should be used sparingly; This depends on your use case and how much data you are working with

#### **Action Item - Writing a query using `UNION`**

The restaurant owners tell us that they are planning on making some changes to the menu next month. They want to replace all of the appetizers and desserts in the current menu with new ones from the `NewDishes` table.

Write a query that **shows the new menu** by combining the `Appetizer` and `Dessert` dishes from `NewDishes` with the `Main` and `Beverage` dishes in the `Dishes` table. Include only the name of the dish, its price, and the type that it is.

<details>
<summary>Click to reveal the expected output</summary>
<img src="https://github.com/freestackinitiative/coop_sql_notebooks/blob/v2/assets/union-action-item-result-102.png?raw=1" alt="Expected Results from Union Query"/>
</details>

<br />

<details>
<summary>Here's a hint if you're stuck</summary>
<p>You can use the previous example as a base to start with since it has almost everything you need. Consider how you might use the <b>IN</b> keyword to help you filter for more than one value in a range efficiently.</p>
</details>

<br />

<details>
<summary>Click here to reveal the answer</summary>

```sql
SELECT Name, Price, Type
FROM NewDishes
WHERE Type IN ('Appetizer', 'Dessert')
UNION
SELECT Name, Price, Type
FROM Dishes
WHERE Type IN ('Main', 'Beverage')
```

</details>

In [4]:
%%sql

SELECT Name, Price, Type
FROM NewDishes
WHERE Type IN ('Appetizer', 'Dessert')
UNION
SELECT Name, Price, Type
FROM Dishes
WHERE Type IN ('Main', 'Beverage')

,Name,Price,Type
0,Zucchini Fritters,9.00,Appetizer
1,Stuffed Bell Peppers,7.00,Appetizer
2,Pumpkin Hummus Platter,9.99,Appetizer
3,Banana Nut Bread Pudding,9.00,Dessert
4,Mini Fruit Tarts,6.00,Dessert
5,Strawberry Rhubarb Pie,5.00,Dessert
6,French Onion Soup,7.00,Main
7,Garden Buffet,9.99,Main
8,Chef's Salad,9.00,Main
9,Quinoa Salmon Salad,9.99,Main


### **Combining Data <em>Horizontally</em> with `JOIN`**

Great work so far! By learning how to combine data vertically using `UNION` you have greatly expanded the number and types of questions you can answer using SQL. Let's take another step further together and learn how to combine data *horizontally* with joins!

Here is a visual that illustrates how joins combine data:

![Simple Join Visual](https://github.com/freestackinitiative/coop_sql_notebooks/blob/v2/assets/simple-join.png?raw=1)

As you can see, the data from `Table A` is combined with `Table B` such that we add the columns from `Table B` onto `Table A`. We're adding data in the *horizontal* direction as opposed to the *vertical* when we use `UNION`.

#### `JOIN` Terminology: The "Left-Side" and "Right-Side" of the join

When discussing joins in SQL, the terms "left-side" and "right-side" refer to the positions of the tables being joined relative to the JOIN keyword in the SQL query.

**Left-Side**

The table that is mentioned immediately after the `FROM` clause is considered the table on the "left-side" of the join. This is the primary table you start with.

**Right-Side**

The table that is specified immediately after the `JOIN` keyword is considered the table on the "right-side" of the join. This is the table you want to join with the left-side table.

#### **The Four Fundamental Types of `JOIN`**

There are four fundamental join types that you should know about in SQL - `INNER`, `LEFT`, `RIGHT`, and `FULL OUTER` joins. Each join type provides slightly different results, and you select the type to use based on what your analysis calls for.

- `INNER JOIN`: Only keep the records that match the constraint between the two tables

- `LEFT JOIN`: Keep all the records from the left-side of the join, and only show values for the records on the right-side that matched. Any values from the right-side that weren’t matched will be assigned a null value.

- `RIGHT JOIN`: The opposite of the `LEFT JOIN` - keep all the records from the right-side of the join and only show values for the records on the left-side that matched. Any values from the left-sie that weren’t matched will be assigned a null value.

- `FULL OUTER JOIN`: Keep all records between both tables, but only show the values that match my constraint. All other records that don’t match will be included, but those values will be set to null.

We will cover how to use each type of join in the following few sections of this notebook.

#### **`INNER JOIN` - The Most Common Join**

When we use an `INNER JOIN`, we are saying, **<em>"Only give me the rows from both tables that match the constraints that I set."</em>**

It can be useful to visualize how the rows will be matched between two tables:

![Inner Join Example](https://github.com/freestackinitiative/coop_sql_notebooks/blob/v2/assets/simple-inner-join.png?raw=1)
![Inner Join](https://www.codeproject.com/KB/database/Visual_SQL_Joins/INNER_JOIN.png)


##### **Example - `INNER JOIN`**

For example, let's say that we want to know our customer's favorite dishes. We only want to see results where the customer has indicated what their favorite dish is - if they didn't tell us, then we're not concerned (for now.) This is a perfect use case for an `INNER JOIN`.  

Let's write a query that identifies each customer's favorite dish from our menu. We'll use data from the `Dishes` table as well as the `Customers` table:

```sql
SELECT C.FirstName, C.LastName, D.Name AS FavoriteDish
FROM Customers AS C
INNER JOIN Dishes AS D -- Specifying the join type
ON C.FavoriteDish=D.DishID -- Specifying our constraint: Rows must have matching DishID numbers
LIMIT 5 -- Limiting to just the first five rows for now because there are a lot of rows
```

##### **Breakdown - `INNER JOIN`**

Let's go through this query step-by-step:

##### **Using aliases when combining multiple tables**

```sql
SELECT C.FirstName, C.LastName, D.Name AS FavoriteDish
FROM Customers AS C
```

The first part of our query is pretty straightforward - even if you might see some unfamiliar notation - because it is a `SELECT` query.

The reason you see `C.FirstName` and such with the columns is because we have created aliases for each table: `C` for `Customers` and `D` for `Dishes`. This is common practice when we write join queries.

If two tables have columns with the same name, SQL needs a way to understand which table and columns you are referring to in your query. This is why we use `C.FirstName`, which is using the `FirstName` column from the `Customers` table.

##### **Specifying the Join Type and Constraint**

```sql
INNER JOIN Dishes AS D
ON C.FavoriteDish=D.DishID
```

This is where we indicate the type of join we want and our intent to join the `Dishes` table. We have specified an `INNER` join as the type of join that we want to use. We gave the `Dishes` table an alias of `D` to make it easier to reference.

Our constraint is specified on the next line. Here, our constraint says, <em>“Only match the rows between the two tables that have matching dish IDs.”</em> Or in other words, <em>“make sure the customer's favorite dishes are associated with the correct dish using the `DishID` number.”</em>

It is very common to use ID fields for joins since they usually represent a primary key/foreign key relationship.

##### **Action Item - Using `INNER JOIN`**

Write a query that shows the names of all the dishes for `OrderID` numbers `1` through `5`. Use the `OrderID` column from the `OrdersDishes` table, and get the dish names from the `Dishes` table. Both of these tables can be connected using a common `DishID`. Your query should just show the `OrderID` and the `Name` of the dish. Order the results by `OrderID`.

>**Pro-Tip:** Don't be afraid to try parts of your query separately or to write it out in stages until you can piece it all together - there is no rule that says you have to get it right all at once. Breaking a big problem down into smaller sub-problems is the essence of coding.

<details>
<summary>Click to reveal the expected output</summary>
<img src="https://github.com/freestackinitiative/coop_sql_notebooks/blob/v2/assets/inner-join-action-item.png?raw=1" alt="An image showing the results of the join query for this action item">
</details>

<br/>
<details>
<summary>Here's a hint if you need it</summary>
<p>Structure the join portion of the query just like the example above. Once you get that, add in the filter for the correct OrderIDs and set the sort order.</p>
</details>

<br/>
<details>
<summary>Click here to reveal the answer</summary>

```sql
SELECT OD.OrderID, D.Name AS DishName
FROM OrdersDishes AS OD
INNER JOIN Dishes AS D
ON OD.DishID=D.DishID
WHERE OD.OrderID BETWEEN 1 AND 5
ORDER BY OD.OrderID
```

</details>

In [20]:
%%sql

-- Putting my expected output
SELECT
  OD.OrderID,
  D.Name AS DishName
FROM OrdersDishes AS OD -- Naming 'OrdersDishes' table

-- Joining the table
INNER JOIN Dishes AS D -- Specifying the join type (adding/naming 'Dishes' table)
ON OD.DishID=D.DishID -- Matching DishID numbers

-- Constraint for both tables
WHERE OD.OrderID BETWEEN 1 AND 5 -- Filter for both tables
ORDER BY OrderID

,OrderID,DishName
0,1,Cheesecake
1,1,Apple Pie
2,2,Mini Cheeseburgers
3,2,Quinoa Salmon Salad
4,3,Pomegranate Iced Tea
5,3,Chocolate Chip Brownie
6,4,Panko Stuffed Mushrooms
7,5,Mixed Berry Tart
8,5,Creme Brulee
9,5,Fiesta Family Platter


#### **`LEFT JOIN` - The Other Most Common Join**

Now that you know how the `INNER JOIN` works, let's introduce you to the other most common join type: `LEFT JOIN`. Thankfully, we don't really need to learn any more new syntax for the rest of the joins. For example, we can simply replace `INNER` with `LEFT` in the previous query to convert it from an `INNER JOIN` to a `LEFT JOIN`.

From here on out, the **`JOIN` syntax is not as important as understanding how your choice of join type affects what rows are returned from the query.**

**`LEFT JOIN` keeps all the records from the left-side of the join and only shows values for the records on the right-side that matched. Any values from the right-side that weren’t matched will be assigned a null value.**

Check out the visual example below:

![Simple Left Join](https://github.com/freestackinitiative/coop_sql_notebooks/blob/v2/assets/simple-left-join.png?raw=1)
![Simple Left Join Venn Diagram](https://www.codeproject.com/KB/database/Visual_SQL_Joins/LEFT_JOIN.png)

As you can see, since there isn't a match for `ID` number `4` in the left-side table, the values from that row are not included in the final result. Instead, we have `NULL` values occupying the space where those values would have gone if there was a match.

##### **Example - Using `LEFT JOIN`**

When we used `INNER JOIN` to find our customer's favorite dishes earlier, did we include *ALL* of the customers in our result? Think about this for a moment before revealing the answer below:

<details>
<summary><em>Well, did we include all of them?</em></summary>
<p><b>No, we did not!</b> Since we used an <b>INNER JOIN</b>, only the customers who had a favorite dish listed were included in the results. But there are customers in our table who don't have a favorite dish listed yet!</p>
</details>

Let's take our original query, change `INNER` to `LEFT`, and check for `NULL` values:

```sql
SELECT C.FirstName, C.LastName, D.Name AS FavoriteDish
FROM Customers AS C
LEFT JOIN Dishes AS D -- Specifying the join type
ON C.FavoriteDish=D.DishID -- Specifying our constraint: Rows must have matching DishID numbers
WHERE C.FavoriteDish IS NULL
```

`Output`:

![Result of Left Join example query](https://github.com/freestackinitiative/coop_sql_notebooks/blob/v2/assets/left-join-results-customer-dishes-example.png?raw=1)

##### **Breakdown - Using `LEFT JOIN`**

There is not much new material to introduce for this query. We are re-using our query from the `INNER JOIN` section and just switched `INNER` to `LEFT`. Then we used a filter to select rows where the the `FavoriteDish` column from `Customers` is `NULL` (`WHERE C.FavoriteDish IS NULL`).

What this does illustrate, though, is a key difference between `INNER` and `LEFT` joins. Originally, we only wanted to know about the customers who specified their favorite dish, so we used an `INNER JOIN`. But if we wanted to also consider the customers who didn't specify their favorite dish yet (maybe so we can reach out to them and ask), then we would want to choose the `LEFT JOIN`.

##### **Action Item - Using `LEFT JOIN`**

Let's revisit the query that we wrote in the previous **Action Item** for `INNER JOIN`. We might have missed some parts of those orders because our first query only considered the rows where the `DishID` matched. Write a version of that query that uses a `LEFT JOIN` instead to display all of the rows for `OrderID` numbers 1-5. Use the same tables - `Dishes` and `OrderDishes`.

Did we miss some dishes in our original (`INNER JOIN`) query?

<details>
<summary>Click to see the expected output</summary>
<img src="https://github.com/freestackinitiative/coop_sql_notebooks/blob/v2/assets/left-join-action-item.png?raw=1" alt="Left Join results for the action item query">
</details>

<br/>
<details>
<summary>Click to reveal the answer</summary>

Yes, we did miss some dishes in the original query. Here is the new query that shows the rows we missed:

```sql
SELECT OD.OrderID, D.Name AS DishName
FROM OrdersDishes AS OD
LEFT JOIN Dishes AS D
ON OD.DishID=D.DishID
WHERE OD.OrderID BETWEEN 1 AND 5
ORDER BY OD.OrderID
```

</details>




In [ ]:
%%sql




#### **`INNER JOIN` vs `LEFT JOIN` - Caveats & When To Use Each Type**

As we mentioned earlier, `INNER JOIN` and `LEFT JOIN` are by far the two most common joins you will use in practice. You should know these two join types well - especially for technical interviews.

The other joins (`RIGHT` and `FULL OUTER`) are not used much in practice, but you should know how they work.

Here is some general advice on deciding which one you should use and caveats to both:

##### **Caveats of using `INNER JOIN` and `LEFT JOIN`**

**`INNER JOIN`:**
- Data Loss: `INNER JOIN` eliminates rows from both tables that do not satisfy the join condition, which could result in data loss if you're interested in those unmatched rows.

**`LEFT JOIN`:**
- `NULL` Values: `LEFT JOIN` retains all rows from the left table, even if there are no matches in the right table, filling these with `NULL` values. This could complicate data analysis or require additional data cleaning steps.

##### **When do I use one over the other?**

Whether to use an `INNER JOIN` or a `LEFT JOIN` is something you must consider for your particular use case. Do you only want to consider the records that match between your tables? Then choose an `INNER JOIN`. Want to make sure that records are kept from the left side of the join? Then - you guessed it - use a `LEFT JOIN`.

---

#### **Lab 1 - `RIGHT JOIN` and `FULL OUTER JOIN`**

In this lab, you will learn about `RIGHT JOIN` and `FULL OUTER JOIN`, as well as their use cases.

**Instructions**

Read through each section and complete the associated **Action Item** tasks for each one.

#### `RIGHT JOIN` and `FULL OUTER JOIN` - Use Cases

We have already mentioned that neither `RIGHT JOIN` or `FULL OUTER JOIN` are used much in practice. This is especially true for `RIGHT JOIN` because we can accomplish the same thing that a `RIGHT JOIN` does with a `LEFT JOIN` - so it's a bit redundant.

`FULL OUTER JOIN` can be useful when you have two or more similar tables (meaning they have the same kind of data, like Domestic Sales and International Sales) and you want to see all of the rows from both together. It can also be helpful when doing a gap analysis where you're trying to find what is missing between two tables.

Regardless of their limited use in practice, you should know how both `RIGHT JOIN` and `FULL OUTER JOIN` work. They are commonly asked about in technical interviews, so you'll need to be aware of them at least. And you never know when one of these will come in handy for a query that you're writing.

##### **`RIGHT JOIN` - How it works**

The opposite of the `LEFT JOIN`, `RIGHT JOIN` includes all the rows from the “right-side” of the join and only rows that match from the “left-side”. Also, similar to `LEFT JOIN`, values in rows from the other side of the join that don’t match are set to null and included in our query results.

Here is an example query:

```sql
SELECT *
FROM A -- This is the table on the "left-side" of the join
RIGHT JOIN B -- This is the table on the "right-side" of the join
ON A.ID=B.ID -- Specifying the join column (`column1`) and constraint (`=`)
```

And this is a visual representation of the results from the query:

![Right Join Simple Example](https://github.com/freestackinitiative/coop_sql_notebooks/blob/v2/assets/simple-right-join.png?raw=1)
![Right Join Simple Example](https://www.codeproject.com/KB/database/Visual_SQL_Joins/RIGHT_JOIN.png)

As you can see, the row on the "left-side" of the join that doesn't have a match in the table on the "right-side" is excluded in the final query result. In the result, any rows on the "right-side" without a match on the "left-side" have `NULL` values in the space where the "left-side" columns should be.

##### **Action Item - Using `RIGHT JOIN` in a query**

Take the query that you wrote for the `LEFT JOIN` **Action Item** earlier and change it so that it uses a `RIGHT JOIN` instead. The results should be the same as what you got using the `LEFT JOIN`.

<details>
<summary>Click to see the expected output</summary>
<img src="https://github.com/freestackinitiative/coop_sql_notebooks/blob/v2/assets/left-join-action-item.png?raw=1" alt="Left Join results for the action item query">
</details>

<br/>
<details>
<summary>Here's a hint...</summary>
<p>Think about the "left-side" and "right-side" of the join - in order to have this query show the same output for a RIGHT JOIN, you'll need to switch the positions of the tables in the query</p>
</details>

<br/>
<details>
<summary>Click to reveal the answer</summary>

```sql
SELECT OD.OrderID, D.Name AS DishName
FROM Dishes AS D
RIGHT JOIN OrdersDishes AS OD
ON OD.DishID=D.DishID
WHERE OD.OrderID BETWEEN 1 AND 5
ORDER BY OD.OrderID
```

</details>

In [ ]:
%%sql



#### **`FULL OUTER JOIN` - How it works**

We use `FULL OUTER JOIN` when we want to include all the rows from both sides of the join, showing the rows that match between the two and otherwise giving null values where there isn’t a match between the tables. A `FULL OUTER JOIN` is like a combination of both the `LEFT` and `RIGHT` join types.

Here is an example of a query using `FULL OUTER JOIN`:

```sql
SELECT *
FROM A -- This is the table on the "left-side" of the join
FULL OUTER JOIN B -- This is the table on the "right-side" of the join
ON A.ID=B.ID -- Specifying the join column (`column1`) and constraint (`=`)
```
Here is how the rows match visually:

![Full Outer Join Simple Example](https://github.com/freestackinitiative/coop_sql_notebooks/blob/v2/assets/simple-full-join.png?raw=1)
![Full Outer Join venn diagram](https://www.codeproject.com/KB/database/Visual_SQL_Joins/FULL_OUTER_JOIN.png)

As shown here, the `FULL OUTER JOIN` combines all the rows from both tables, showing `NULL` values in rows where there isn't a match in the other table based on the constraint. However, all of the rows are there in the result.

##### **Action Item - Using `FULL OUTER JOIN`**

**Scenario**

The restaurant is preparing for some events that customers have made reservations for. To keep track of everything that needs to be prepared for each event and make sure all the dishes can be prepared, they used the `CustomersDishes` and `CustomersEvents` tables to store information. Unfortunately, they just found out that the software they were using to keep track of these events had unexpected errors causing some data to get corrupted and not stored correctly. They need your help to find the entries that are missing data so they can make sure everything goes smoothly for their events!

**Your Task**

Usually, if we join the `CustomersEvents` table with the `CustomersDishes` table using the `CustomerID`, there should be no `NULL` values. But since you know about the software error, you figure there will probably be `NULL` values where the entries were lost. Use a `FULL OUTER JOIN` to help them identify the rows that were corrupted from both tables and save the day!

Your output should only show the `EventID` from the `CustomersEvents` table and the `CustomerID` from the `CustomersDishes` table.

<details>
<summary>Click here to see the expected output</summary>
<img src="https://github.com/freestackinitiative/coop_sql_notebooks/blob/v2/assets/full-outer-join-action-item.png?raw=1" alt="Full outer join results for action item">
</details>

<br/>
<details>
<summary>Click here for a hint!</summary>
<p>Use the FULL OUTER JOIN example from above to help you get started. Change the table names to match the ones in the task, then update your contraints based on what the question is asking.</p>
</details>

<br/>
<details>
<summary>Click here to reveal the answer</summary>

```sql
SELECT CE.EventID, CD.CustomerID
FROM CustomersEvents CE
FULL OUTER JOIN CustomersDishes CD
ON CE.CustomerID=CD.CustomerID
```

</details>


In [ ]:
%%sql



**You've finished Lab 1! Great work!**

---

### **<em>Comprehension Check</em> - Combining Data Using `JOIN` and `UNION`**

Answer the questions below to check your understanding of what we have covered so far. Try to answer the questions first before looking at the answers:

*1. What is the primary difference between joins and unions in SQL?*
<details>
    <summary>Click to reveal the answer</summary>
    <p>Joins combine data horizontally by adding more columns, while unions combine data vertically by adding more rows.</p>
</details>


*2. What is the key difference between `INNER JOIN` and `LEFT JOIN`?*
<details>
    <summary>Click to reveal the answer</summary>
    <p>`INNER JOIN` keeps only the records that match the constraint between the two tables, while `LEFT JOIN` keeps all the records from the left-side table and only the matching records from the right-side table.</p>
</details>


*3. What does `RIGHT JOIN` do?*
<details>
    <summary>Click to reveal the answer</summary>

`RIGHT JOIN` is the opposite of `LEFT JOIN`. It keeps all the records from the right-side table and only the matching records from the left-side table. Unmatched records from the left-side table will have `NULL` values.

</details>

*4. Describe the difference between `UNION ALL` and `UNION.`*
<details>
    <summary>Click to reveal the answer</summary>
    <p>`UNION ALL` includes all results, even if there are duplicate rows. `UNION` removes all duplicate rows from the result set.</p>
</details>

### **Summary: Combining Data with Joins and Union**

We use both joins and unions to combine data. Joins let us combine data *horizontally* by adding more columns. Unions, on the other hand, let us combine data *vertically* by adding more rows.

Overall, there are **four join types** that you should know well:

- `INNER JOIN`: Only keep the records that match the constraint between the two tables

- `LEFT JOIN`: Keep all the records from the left-side of the join, and only show values for the records on the right-side that matched. Any values from the right-side that weren't matched will be assigned a `null` value.

- `RIGHT JOIN`: The opposite of the `LEFT JOIN` - keep all the records from the right-side of the join and only show values for the records on the left-side that matched. Any values from the left-sie that weren't matched will be assigned a `null` value.

- `FULL OUTER JOIN`: Keep all records between both tables, but only show the values that match my constraint. All other records that don't match will be included, but those values will be set to null.

There are **two types of `UNION` queries** you should know:
- `UNION ALL`: All results from tables in the `UNION` are included, even if there are duplicate rows
- `UNION`: Removes all duplicate rows from the `UNION` of the tables

The join types mentioned here are the most important to know, but there are a few more join types that you may come across. There are some more advanced joins, like [cross-joins, natural joins, and self-joins](https://www.linkedin.com/pulse/what-difference-between-natural-joincross-join-self-madhu-mitha-k) that you should eventually become familiar with as you enhance your skills and understanding.

## Conclusion

Congratulations on completing SQL 103! You have come a long way from your humble beginnings in SQL 101 and you've developed some awesome querying skills. Great work!

As always, we encourage you to continue practicing and learning more. Use both the resources we provide and find opportunities to learn independently.

Now you are ready to help us solve the [**SQL Murder Mystery!**](https://colab.research.google.com/github/freestackinitiative/coop_sql_notebooks/blob/v2/COOP_SQL_Murder_Mystery.ipynb)

## More resources for further practice

- [SQL Bolt](https://sqlbolt.com/): The lessons here are a great introduction to SQL and you know the platform already!
- [Mode](https://mode.com/sql-tutorial/): A comprehensive SQL tutorial from beginner all the way to advanced SQL. There's even a data analytics with SQL tutorial. This is a great resource to learn about SQL in depth and practice what you learn in their online database.
- [StrataScratch](https://platform.stratascratch.com/coding): Practice coding questions geared toward data analysts and data scientists. You can solve coding problems used by real companies for technical interviews using PostgresSQL, Python, R, or MySQL. It's free to sign up!
- [Codecademy - Free Learn SQL Course](https://www.codecademy.com/learn/learn-sql): Codecademy is another great resource to learn SQL as well as most other languages. There are a lot of free resources here that can help you learn SQL, Python, R, and many other languages.
- [Socratica SQL (YouTube)](https://www.youtube.com/watch?v=nWyyDHhTxYU&list=PLih4ch-U2DiBbMoFK4ML9faT3k3MM2UQY): This is a great playlist that will get you started learning SQL with one of the most popular relational databases - Postgres.
- [DB Fiddle](https://dbfiddle.uk/): This site is like a SQL scratch pad. You can use it to practice doing stuff like creating tables and inserting data into them, and all sorts of other stuff that you might not be able to do so freely in a live database. It's a sandbox, basically. Here are a couple of links to fiddles with some data in them to play with: [fiddle 1](https://dbfiddle.uk/?rdbms=postgres_13&fiddle=366b683701596d3f7459b0411c15acd1) and [fiddle 2](https://dbfiddle.uk/?rdbms=postgres_13&fiddle=dfffc1939f629d9286c55d732fb656c5).


And don't forget to keep your [SQL Cheatsheet](https://martinmarroyo.github.io/sqlcheatsheetandresources-coop/) handy!